<a href="https://colab.research.google.com/github/ProfeLuisTic1986/Tutoriales/blob/main/Chatbot_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial 1: Chatbot Básico con Ollama y Flet (sin voz)**

¿Qué vas a crear?
Un chatbot con interfaz gráfica que se conecta a Ollama para responder como Albert Einstein (Mac) o Marie Curie (Windows).

**Paso 1: Preparación del entorno**

1.1. Instala Python
Descarga Python desde python.org
Durante la instalación, marca "Add Python to PATH"

1.2. Instala Ollama
Windows: Descarga desde ollama.com
Mac: En terminal: brew install ollama

1.3. Descarga un modelo

In [ ]:
ollama pull mistral:7b

1.4. Instala las librerías de Python

In [ ]:
pip install flet requests

Paso 2: Estructura del pryecto
2.1. Crea el proyecto App009-Chatbot-ollama

ingresa al archivo main.py

In [ ]:
flet create App009-Chatbot-ollama

2.2. Importa las librerías necesarias

In [ ]:
import flet as ft
import requests
import json
import platform

¿Qué hace cada import?

* flet as ft: Librería para crear la interfaz gráfica
* requests: Para hacer peticiones HTTP a Ollama
* json: Para manejar las respuestas JSON de Ollama
* platform: Para detectar si es Windows, Mac o Linux

2.3. Define las constantes del sistema

In [ ]:
# Detecta el sistema operativo
SO = platform.system()

if SO == "Darwin":  # Mac
    PERSONAJE = "Albert Einstein"
    EMOJI_PERSONAJE = "🧑‍🔬"
elif SO == "Windows":
    PERSONAJE = "Marie Curie"
    EMOJI_PERSONAJE = "👩‍🔬"
else:
    PERSONAJE = "Científico"
    EMOJI_PERSONAJE = "🧑‍🔬"

EMOJI_USUARIO = "🧑‍💻"
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL = "mistral:7b"

¿Qué hace este código?

* platform.system(): Devuelve "Darwin" (Mac), "Windows" o "Linux"
* Según el sistema, cambia el personaje y emoji
* Define la URL donde está corriendo Ollama (siempre localhost:11434)
* Define qué modelo usar (mistral:7b)

2.4. Crea la función principal de la app

In [ ]:
def main(page: ft.Page):
    # Configuración de la ventana
    page.title = f"Chat con {PERSONAJE}"
    page.bgcolor = ft.Colors.GREY_100

¿Qué hace este código?

* def main(page: ft.Page):: Función principal que Flet ejecutará
* page.title: Título de la ventana (cambia según el personaje)
* page.bgcolor: Color de fondo de la ventana (gris claro)

2.5. Crea la lista de mensajes

In [ ]:
# Lista donde se mostrarán los mensajes del chat
    mensajes = ft.ListView(
        expand=True,      # Ocupa todo el espacio disponible
        spacing=10,       # Espacio entre mensajes
        padding=20,       # Margen interno
        auto_scroll=True, # Se desplaza automáticamente al último mensaje
    )

¿Qué hace este código?

* ft.ListView: Un contenedor que muestra una lista de elementos
* expand=True: Hace que ocupe todo el espacio vertical disponible
* spacing=10: Pone 10 píxeles de separación entre cada mensaje
* auto_scroll=True: Cuando hay un mensaje nuevo, se desplaza automáticamente

2.6. Crea la función para mostrar burbujas de chat

In [ ]:
def burbuja(texto, es_usuario):
        return ft.Row(
            [
                # Emoji del usuario o del personaje
                ft.Text(EMOJI_USUARIO if es_usuario else EMOJI_PERSONAJE, size=24),
                # Contenedor con el texto del mensaje
                ft.Container(
                    content=ft.Text(
                        texto,
                        color=ft.Colors.WHITE if es_usuario else ft.Colors.BLACK,
                        size=15,
                        selectable=True,  # El usuario puede seleccionar el texto
                    ),
                    # Color de fondo: azul para usuario, gris para bot
                    bgcolor=ft.Colors.BLUE_400 if es_usuario else ft.Colors.GREY_300,
                    padding=12,           # Espacio interno
                    border_radius=30,     # Bordes redondeados
                    # Sombra para dar efecto 3D
                    shadow=ft.BoxShadow(blur_radius=8, color=ft.Colors.GREY_400, offset=ft.Offset(2, 2)),
                    # Margen: izquierda para usuario, derecha para bot
                    margin=ft.margin.only(left=10) if es_usuario else ft.margin.only(right=10),
                    width=350,            # Ancho máximo de la burbuja
                )
            ],
            # Alineación: derecha para usuario, izquierda para bot
            alignment=ft.MainAxisAlignment.END if es_usuario else ft.MainAxisAlignment.START,
        )

¿Qué hace este código?

* def burbuja(texto, es_usuario):: Función que crea una burbuja de mensaje
* es_usuario: Parámetro booleano (True si es del usuario, False si es del bot)
* ft.Row: Organiza elementos horizontalmente (emoji + burbuja)
* ft.Container: Caja que contiene el texto con estilo
* Los colores, márgenes y alineación cambian según quién envía el mensaje




2.7. Crea el campo de texto para escribir

In [ ]:
# Campo donde el usuario escribe su mensaje
    prompt = ft.TextField(
        label="Escribe tu mensaje...",  # Texto de ayuda
        expand=True,                    # Ocupa todo el ancho disponible
        border_radius=20,               # Bordes redondeados
        filled=True,                    # Tiene fondo
        bgcolor=ft.Colors.WHITE         # Fondo blanco
    )

¿Qué hace este código?

* ft.TextField: Campo de texto donde el usuario puede escribir
* label: Texto que aparece cuando el campo está vacío
* expand=True: Hace que ocupe todo el ancho disponible
* border_radius=20: Hace los bordes redondeados
* filled=True y bgcolor: Le da un fondo blanco

2.8. Crea la función para enviar mensajes

In [ ]:
def enviar_click(e):
        # Obtiene el texto que escribió el usuario
        user_input = prompt.value.strip()
        if not user_input:  # Si está vacío, no hace nada
            return

        # Agrega el mensaje del usuario a la lista
        mensajes.controls.append(burbuja(user_input, es_usuario=True))
        page.update()  # Actualiza la interfaz

        # Limpia el campo de texto
        prompt.value = ""
        page.update()

        # Crea el prompt para el personaje
        prompt_personaje = (
            f"Responde como si fueras {PERSONAJE}. "
            "Habla con su estilo, conocimientos y personalidad. "
            "Responde en español de manera clara y concisa. "
            f"Pregunta del usuario: {user_input}"
        )

        try:
            # Envía la petición a Ollama
            response = requests.post(
                OLLAMA_URL,
                json={"model": MODEL, "prompt": prompt_personaje, "stream": True},
                stream=True
            )

            respuesta = ""
            # Lee la respuesta línea por línea
            for line in response.iter_lines():
                if line:
                    data = json.loads(line)
                    if "response" in data:
                        respuesta += data["response"]
                    elif "error" in data:
                        respuesta = f"Error de Ollama: {data['error']}"
                        break

            if not respuesta:
                respuesta = "No se recibió respuesta del modelo."

        except Exception as ex:
            respuesta = f"Error de conexión: {ex}"

        # Agrega la respuesta del bot a la lista
        mensajes.controls.append(burbuja(respuesta, es_usuario=False))
        page.update()

¿Qué hace este código?

* def enviar_click(e):: Función que se ejecuta al hacer clic en "Enviar"
* prompt.value.strip(): Obtiene el texto y quita espacios en blanco
* mensajes.controls.append(): Agrega un nuevo mensaje a la lista
* page.update(): Actualiza la interfaz para mostrar los cambios
* requests.post(): Envía una petición HTTP a Ollama
* stream=True: Recibe la respuesta en tiempo real
* json.loads(): Convierte el texto JSON en un diccionario de Python
* El try/except maneja errores de conexión

2.9. Conecta el campo de texto con la función

In [ ]:
# Cuando el usuario presiona Enter, ejecuta enviar_click
    prompt.on_submit = enviar_click

¿Qué hace este código?

* on_submit: Evento que se dispara cuando el usuario presiona Enter
* Conecta el campo de texto con la función de enviar

2.10. Crea el encabezado y organiza la interfaz

In [ ]:
# Encabezado del chat
    header = ft.Container(
        content=ft.Row([
            ft.Text(EMOJI_PERSONAJE, size=32),  # Emoji grande
            ft.Text(PERSONAJE, size=22, weight="bold", color=ft.Colors.BLUE_900),
        ], alignment=ft.MainAxisAlignment.START, spacing=15),
        padding=ft.padding.symmetric(vertical=16, horizontal=10),
        bgcolor=ft.Colors.WHITE,
        border_radius=ft.border_radius.only(top_left=20, top_right=20),
        shadow=ft.BoxShadow(blur_radius=12, color=ft.Colors.GREY_300, offset=ft.Offset(0, 2))
    )

    # Organiza todos los elementos en la página
    page.add(
        ft.Container(
            content=ft.Column([
                header,           # Encabezado
                mensajes,         # Lista de mensajes
                ft.Row([          # Fila con campo de texto y botón
                    prompt,
                    ft.ElevatedButton(
                        "Enviar",
                        on_click=enviar_click,
                        bgcolor=ft.Colors.BLUE_400,
                        color=ft.Colors.WHITE
                    )
                ], vertical_alignment=ft.CrossAxisAlignment.END),
            ], expand=True, spacing=10),
            expand=True,
            padding=0,
            bgcolor=ft.Colors.WHITE,
        )
    )

¿Qué hace este código?

* header: Crea un encabezado bonito con el emoji y nombre del personaje
* ft.Column: Organiza elementos verticalmente (encabezado, mensajes, campo de texto)
* ft.Row: Organiza elementos horizontalmente (campo de texto + botón)
* page.add(): Agrega todos los elementos a la página

2.11. Ejecuta la aplicación

Paso 3: Prueba tu chatbot básico
* Guarda tu proyecto
* Asegúrate de que Ollama esté corriendo: ollama serve
* Ejecuta: tu archivo main.py
* ¡Chatea con Albert Einstein (Mac) o Marie Curie (Windows)!

Codigo completo

In [ ]:
import flet as ft
import requests
import json

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL = "mistral:7b"
PERSONAJE = "Albert Einstein"
EMOJI_PERSONAJE = "🧑‍🔬"
EMOJI_USUARIO = "🧑‍💻"

def main(page: ft.Page):
    page.title = f"Chat con {PERSONAJE}"
    page.bgcolor = ft.Colors.GREY_100

    mensajes = ft.ListView(
        expand=True,
        spacing=10,
        padding=20,
        auto_scroll=True,
    )

    def burbuja(texto, es_usuario):
        return ft.Row(
            [
                ft.Text(EMOJI_USUARIO if es_usuario else EMOJI_PERSONAJE, size=24),
                ft.Container(
                    content=ft.Text(
                        texto,
                        color=ft.Colors.WHITE if es_usuario else ft.Colors.BLACK,
                        size=15,
                        selectable=True,
                    ),
                    bgcolor=ft.Colors.BLUE_400 if es_usuario else ft.Colors.GREY_300,
                    padding=12,
                    border_radius=30,
                    shadow=ft.BoxShadow(blur_radius=8, color=ft.Colors.GREY_400, offset=ft.Offset(2, 2)),
                    margin=ft.margin.only(left=10) if es_usuario else ft.margin.only(right=10),
                    alignment=ft.alignment.center_right if es_usuario else ft.alignment.center_left,
                    width=350,
                )
            ] if es_usuario else [
                ft.Container(
                    content=ft.Text(
                        texto,
                        color=ft.Colors.BLACK,
                        size=15,
                        selectable=True,
                    ),
                    bgcolor=ft.Colors.GREY_300,
                    padding=12,
                    border_radius=30,
                    shadow=ft.BoxShadow(blur_radius=8, color=ft.Colors.GREY_400, offset=ft.Offset(2, 2)),
                    margin=ft.margin.only(right=10),
                    alignment=ft.alignment.center_left,
                    width=350,
                ),
                ft.Text(EMOJI_PERSONAJE, size=24),
            ],
            alignment=ft.MainAxisAlignment.END if es_usuario else ft.MainAxisAlignment.START,
        )

    prompt = ft.TextField(
        label=f"Escribe tu mensaje...",
        expand=True,
        border_radius=20,
        filled=True,
        bgcolor=ft.Colors.WHITE
    )

    def enviar_click(e):
        user_input = prompt.value.strip()
        if not user_input:
            return
        mensajes.controls.append(burbuja(user_input, es_usuario=True))
        page.update()
        prompt.value = ""
        page.update()

        prompt_personaje = (
            f"Responde como si fueras {PERSONAJE}. "
            "Habla con su estilo, conocimientos y personalidad. "
            f"Pregunta del usuario: {user_input}"
        )

        try:
            response = requests.post(
                OLLAMA_URL,
                json={"model": MODEL, "prompt": prompt_personaje, "stream": True},
                stream=True
            )
            respuesta = ""
            for line in response.iter_lines():
                if line:
                    data = json.loads(line)
                    if "response" in data:
                        respuesta += data["response"]
                    elif "error" in data:
                        respuesta = f"Error de Ollama: {data['error']}"
                        break
            if not respuesta:
                respuesta = "No se recibió respuesta del modelo."
        except Exception as ex:
            respuesta = f"Error de conexión o inesperado: {ex}"

        mensajes.controls.append(burbuja(respuesta, es_usuario=False))
        page.update()

    # Header con emoji y nombre
    header = ft.Container(
        content=ft.Row([
            ft.Text(EMOJI_PERSONAJE, size=32),
            ft.Text(PERSONAJE, size=22, weight="bold", color=ft.Colors.BLUE_900),
        ], alignment=ft.MainAxisAlignment.START, spacing=15),
        padding=ft.padding.symmetric(vertical=16, horizontal=10),
        bgcolor=ft.Colors.WHITE,
        border_radius=ft.border_radius.only(top_left=20, top_right=20),
        shadow=ft.BoxShadow(blur_radius=12, color=ft.Colors.GREY_300, offset=ft.Offset(0, 2))
    )

    page.add(
        ft.Container(
            content=ft.Column([
                header,
                mensajes,
                ft.Row([prompt, ft.ElevatedButton("Enviar", on_click=enviar_click, bgcolor=ft.Colors.BLUE_400, color=ft.Colors.WHITE)], vertical_alignment=ft.CrossAxisAlignment.END),
            ], expand=True, spacing=10),
            expand=True,
            padding=0,
            border_radius=0,
            bgcolor=ft.Colors.WHITE,
        )
    )

ft.app(target=main)